In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append("/Users/danem/Projects/hack-for-social")

import dotenv
dotenv.load_dotenv("/Users/danem/Projects/hack-for-social/secrets/env.local")

import backend.main
import backend.llm
from llama_index.core.node_parser import TokenTextSplitter, SentenceWindowNodeParser, SentenceSplitter
from llama_index.core import SimpleDirectoryReader, Document, StorageContext
from llama_index.readers.file import PDFReader, DocxReader, FlatReader  # type: ignore
import nest_asyncio
from llama_index.core.extractors import (
    TitleExtractor,
    QuestionsAnsweredExtractor,
    SummaryExtractor,
)
from llama_index.core.schema import TextNode
from llama_index.core.vector_stores.types import VectorStoreQueryMode
import collections

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
nest_asyncio.apply()
nodes = backend.llm.load_directory("/Users/danem/Projects/hack-for-social/data/foo")

100%|██████████| 70/70 [00:00<00:00, 42202.28it/s]


In [3]:
vindex = backend.llm.get_vector_index("test3")

In [4]:
vindex.insert_nodes(nodes.nodes)

In [48]:
qe = vindex.as_retriever(similarity_top_k=10, vector_store_query_mode="HYBRID")

ValueError: 'HYBRID' is not a valid VectorStoreQueryMode

In [46]:
m = qe.retrieve("mold")
lut = collections.defaultdict(list)
for x in m:
    if x.score < 0.4:
        continue
    lut[x.metadata["file_name"]].append(x)

In [47]:
lut

defaultdict(list,
            {'California_1556 2023.11.21 HODecision_Redacted.pdf': [NodeWithScore(node=TextNode(id_='5a0f930e-bafd-4317-a79b-860e403ec0c7', embedding=None, metadata={'page_label': '4', 'file_name': 'California_1556 2023.11.21 HODecision_Redacted.pdf', 'file_path': '/Users/danem/Projects/hack-for-social/data/foo/California_1556 2023.11.21 HODecision_Redacted.pdf', 'file_type': 'application/pdf', 'file_size': 532122, 'creation_date': '2024-11-09', 'last_modified_date': '2024-08-05', 'section_summary': 'The key topics in this section include mold issues in the living room, kitchen, and balcony, carpet condition and odor, electrical outlet problems, refrigerator and freezer malfunctions, and leaking shower in the Affected Unit. The entities mentioned are Ms. Zavala, former Landlord, Mr. Sanchez, and the neighbor. Key issues highlighted are long-standing maintenance problems, health concerns related to mold and electrical issues, and food wastage due to refrigerator proble

In [37]:
for y in nodes.keyword_nodes:
    print(y.text)

Mountain View, CSFRA, Rental Property, Hearing Officer Decision, Rent Adjustment
Hearing procedure, Prehearing Meeting, Witness testimony, Post-Hearing Order, Interpretation services
Oralia Belem Zavala Vasquez, rent increases, communication with landlords, maintenance issues, mold problems
mold, maintenance, electrical, refrigerator, shower
hot water leak, stove, exhaust fan, smoke detectors, rat problems
kitchen lights, heating system, repair requests, gas smell, living conditions
water issues, mold, carpet condition, stove functionality, maintenance work
repairs, interactions, stove, electricity, mold
rats, pest control, maintenance, rent increases, property sale
rent increase, property ownership, repairs, maintenance, affidavits
Hearing Officer Exhibits, Petitioner's Exhibits, CSFRA Petition, Rent Adjustment, Inspection Reports
money order, receipts, payments, rental, organizations
money order receipts, entities involved, specific dates, amounts, detailed list
money orders, checks,

In [44]:
len(nodes.nodes)

560